# Task #2

A template code for training an RBM on Rydberg atom data (the full dataset) is provided below. For the first part of this task (determining the minimum number of hidden units), start with 20 hidden units. 

Imports and loadining in data:

In [4]:
from pathlib import Path
import numpy as np
import torch
from RBM_helper import RBM
import csv
import Rydberg_energy_calculator
from datetime import datetime

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))

Define the RBM:

In [5]:
n_vis = training_data.shape[1]


Train the RBM:

In [ ]:

target = 0.0001
exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)
output_dir = Path("training_logs")

if not output_dir.exists():
    output_dir.mkdir(parents=True)
differences = {}
for n_hin in range(1, 200, 10):
    rbm = RBM(n_vis, n_hin)
    epochs = 1000
    num_samples = 2000
    print('Starting training on n_h = ', n_hin)
    differences[n_hin] = []
    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(training_data)

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            # print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            RBM_samples = rbm.draw_samples(100, init_state)

            # print("Done sampling. Calculating energy...")

            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction)
            energy_difference = abs(exact_energy - energies.item())
            differences[n_hin].append(energy_difference)
            print("Energy difference: ", energy_difference)

with open(str(output_dir / f"task2.csv"), 'w') as f:
    w = csv.DictWriter(f, differences.keys())
    w.writeheader()
    w.writerow(differences)

Exact energy:  -4.1203519096
Starting training on n_h =  1


In [3]:
target = 0.0001
exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)
output_dir = Path("training_logs")
if not output_dir.exists():
    output_dir.mkdir(parents=True)
differences = {}
n_hin = 20
for i in range(19):
    train_num = 500 + i * 1000
    print("Training samples: ", train_num)
    sub_training_data = training_data[:train_num]
    rbm = RBM(n_vis, n_hin)
    epochs = 500
    num_samples = 2000
    min_difference = 10000
    min_difference_epoch = 0
    print('Starting training on n_h = ', n_hin)
    differences[n_hin] = []
    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(training_data)

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            # print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            RBM_samples = rbm.draw_samples(100, init_state)

            # print("Done sampling. Calculating energy...")

            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction)
            energy_difference = abs(exact_energy - energies.item())
            differences[n_hin].append(energy_difference)
            print("Energy difference: ", energy_difference)
            if min_difference > energy_difference:
                min_difference = energy_difference
                min_difference_epoch = e
            if  min_difference < target:
                print("Target energy difference obtained at epoch = ", e)
                print("Final energy difference for n_h = ", n_hin, ' : ', energy_difference)
                break


    print("Final energy difference for n_h = ", n_hin, ' : ', min_difference, 'at epoch ', min_difference_epoch)
    if min_difference < target:
        print("Required samples: ", num_samples)
    with open(str(output_dir / f"task2_2_train_num_{train_num}.csv"), 'w') as f:
        w = csv.writer(f)
        w.writerow(differences[n_hin])


Exact energy:  -4.1203519096
Training samples:  500
Starting training on n_h =  20

Epoch:  100
Energy difference:  0.0011855795454716045

Epoch:  200
Energy difference:  0.0003355576834591645

Epoch:  300
Energy difference:  0.0005618488874903349

Epoch:  400
Energy difference:  0.0004804705048417901

Epoch:  500
Energy difference:  0.0009949617058024884
Final energy difference for n_h =  20  :  0.0003355576834591645 at epoch  200
Training samples:  1500
Starting training on n_h =  20

Epoch:  100
Energy difference:  0.0011025590144546626

Epoch:  200
Energy difference:  0.0008292777237821625

Epoch:  300
Energy difference:  0.0006659640594968153

Epoch:  400
Energy difference:  0.0005057504057672801

Epoch:  500
Energy difference:  0.00047566310019586666
Final energy difference for n_h =  20  :  0.00047566310019586666 at epoch  500
Training samples:  2500
Starting training on n_h =  20

Epoch:  100
Energy difference:  0.0008493408702436867

Epoch:  200
Energy difference:  0.001158578

../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
